In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from csnet.training.dataset import get_structure

In [35]:
pdb_file_list =   [
    '/storage_common/angiod/A2A/Vincenzo/FAN.pdb',
    '/storage_common/angiod/A2A/Vincenzo/pAs.pdb',
    '/storage_common/angiod/A2A/Vincenzo/INzma.pdb',
]
traj_files_list = [
    ['/storage_common/angiod/A2A/Vincenzo/FAN_strided.xtc'],
    ['/storage_common/angiod/A2A/Vincenzo/pAs_strided.xtc'],
    ['/storage_common/angiod/A2A/Vincenzo/INzma_strided.xtc'],
]
npy_out_list = [
    '../temp/cs_FAN.npy',
    '../temp/cs_pAs.npy',
    '../temp/cs_INzma.npy',
]

device = "cuda:2"

In [ ]:
from plot import plot_violin, plot_distribution, plot_violindist

atoms_filters = ['ILE_CD1_*',]

dataset, _ = get_structure(pdb_file_list[1])
for atoms_filter in atoms_filters: 
    fltr = np.ones_like(dataset['atom_fullnames'], dtype=bool)
    res_fltr, atom_fltr, resid_fltr = atoms_filter.split('_')
    fltr = fltr * (~np.isin(dataset['atom_resnames'], ['ACE']))
    if res_fltr != '*':
        fltr = fltr * np.isin(dataset['atom_resnames'], res_fltr.split('-'))
    if atom_fltr != '*':
        fltr = fltr * np.isin(dataset['atom_names'].flatten(), atom_fltr.split('-'))
    if resid_fltr != '*':
        fltr = fltr * np.isin(dataset['atom_resnumbers'], [int(x) for x in resid_fltr.split('-')])

    atom_names = dataset['atom_fullnames'][fltr]

    ds_names = npy_out_list # ['../temp/cs_FAN.npy', '../temp/cs_pAs.npy', '../temp/cs_INzma.npy'][::-1]
    cs = []
    for ds_name in ds_names:
        x = np.load(ds_name)
        cs.append(x.reshape(len(x), -1))
    chemical_shifts = [x[:, fltr] for x in cs]

    # plot_distribution(atom_names, chemical_shifts, ds_names, 'Chemical Shifts', figsize=(20,8), overlap_same=False)
    plot_violin(atom_names, chemical_shifts, ds_names, 'Chemical Shifts', overlap_same=True, ylim=None)
    # plot_violindist(chemical_shifts, ds_names, 'Chemical Shifts', figsize=(20,8))